In [1]:
#|default_exp preprocessing
#|export

import polars as pl
import numpy as np

from datetime import datetime
import calendar

import json
import os

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

import torch
from torch.utils.data import DataLoader, TensorDataset 

In [2]:
#|export

class DataHandler():
    '''
    Handles preprocessing, cleaning, and transformation of Lending Club data for downstream machine learning task.
    It provides methods for extracting, transforming, and batching data for model training and evaluation.
    '''
    def __init__(self, csv_path: str = '../local_data/all_lending_club_loan_data_2007-2018.csv'):
        '''
        Initialises the DataHandler.

        - Sets up file paths for the cleaned CSV and feature metadata, and ensures the data is preprocessed and ready for use.
        - If the cleaned data does not exist, it will trigger the cleaning process.
        - Loads feature metadata and initialises the column transformer for feature scaling and encoding.

        Args:
            csv_path (str): Path to the raw Lending Club CSV file.
        '''

        self.cleaned_csv_path = f'{csv_path[:-4]}_cleaned.csv'#Path to cleaned data
        self.features_path = f'{csv_path[:-4]}_cleaned_features.json'#Path to json listing the features in the cleaned dataset, organised by their type

        #If a 'cleaned' version of the data is not found the raw Lending Club data from Kaggle then:
        if not os.path.exists(self.cleaned_csv_path):
            self.strip_non_data_rows_from_lending_club_data(csv_path)#1. Formatting irregularities relating to header and footer rows are removed
            self.clean_lending_club_data()#2.  All variables are processed based on their data type

        #If the clean data is found the data features loaded to initialise the column transformer
        with open(self.features_path, 'r') as f:
            self.features = json.load(f)

        self.transformer = self.column_transformer = ColumnTransformer(
            transformers=[
                ('standard_scaler', StandardScaler(), self.features['standard_scaler']),#Numerical data is standard scaled
                ('min_max_scaler', MinMaxScaler(), self.features['min_max_scaler']),#Ordinal data is scaled using their min and max to maintain the distance between each option 
                ('one_hot_encoder', OneHotEncoder(drop=None, handle_unknown='ignore'), self.features['categorical'] )#Categorical data is one hot encoded to avoid creating untrue patterns between data points
            ],
        )

        #Initialises key values and flags if the training data has been set or not based on whether the values are None
        self.training_data_start_date = None
        self.training_data_end_date = None
    
    def strip_non_data_rows_from_lending_club_data(self, csv_path: str):
        '''
        Removes non-data rows from the raw Lending Club CSV file.

        The Lending Club data may contain header or footer rows that are not part of the actual dataset.
        This function filters out such rows, keeping only those that start with a digit or the header row, and writes the result to a temporary file.

        Args:
            csv_path (str): Path to the raw Lending Club CSV file.
        '''
        with open(csv_path, 'r') as f:
            lines = [line for line in f if line[0].isdigit() or line.startswith('id')]

        with open('temp.csv', 'w') as f:
            f.writelines(lines)#Stores the treated data in a temporary file ready to be processed

    def is_date_value(self, value: str) -> bool:
        '''
        Checks if a string value matches the month-year date format (e.g., 'sep-2015').
        It's used to identify columns that contain date information in the Lending Club dataset.

        Args:
            value (str): The value to check.

        Returns:
            bool: True if the value matches the expected date format, False otherwise.
        '''
        if not isinstance(value, str) or '-' not in value:
            return False
        
        try:
            month_str, year_str = value.lower().split('-')
            return (month_str.title() in calendar.month_abbr and 
                    year_str.isdigit() and 
                    len(year_str) == 4)
        except:
            return False
        
    def parse_date_value(self, value: str) -> datetime:
        '''
        Converts a month-year string (e.g., 'sep-2015') to a datetime object.

        Args:
            value (str): The string to convert.

        Returns:
            datetime: The corresponding datetime object, or None if conversion fails.
        '''
        if not isinstance(value, str):
            return None
        
        try:
            month_str, year_str = value.lower().split('-')
            month_num = list(calendar.month_abbr).index(month_str.title())
            return datetime(int(year_str), month_num, 1)
        except:
            return None
            
    def drop_undesired_columns(self, df: pl.DataFrame)-> pl.DataFrame:
        '''
        Removes columns that are redundant, mostly null, or not useful for modelling.

        This includes explicit columns known to be unhelpful, columns with high null rates, and columns with certain prefixes.

        Args:
            df (pl.DataFrame): The input Polars DataFrame.

        Returns:
            pl.DataFrame: The DataFrame with undesired columns dropped.
        '''
        explicit_columns_to_drop = [
            'id',                       # Unique identifier
            'funded_amnt',              # Redundant due to loan_amnt
            'funded_amnt_inv',          # Redundant due to loan_amnt
            'sub_grade',                # Redundant due to grade column
            'emp_title',                # Too random
            'title',                    # Redundant in relation to purpose column
            'desc',                     # Mostly null
            'url',                      # No predictive value
            'mths_since_last_delinq',   # Mostly null and redundant due to delinq_2yrs column
            'mths_since_last_record',   # Mostly null
            'pymnt_plan',               # Always 'n'
            'addr_state',               # Reduce dimensionality of this excerise
            'zip_code'                  # Reduce dimensionality of this excerise
        ]

        # Dropping all secondary applicant, hardship, settlement, and joint columns as they are all null
        implicit_columns_to_drop = (
            'sec_app_',
            'hardship_',
            'settlement_',
            'joint_',
        )

        implicit_columns_to_drop = [column for column in df.columns if column.startswith(implicit_columns_to_drop)]
        high_null_cols = [column for column in df.columns  if (df[column].null_count() / len(df)) > 0.2]#If  more than 20% of the values are null the column is dropped

        columns_to_drop = explicit_columns_to_drop + implicit_columns_to_drop + high_null_cols
        columns_to_drop = list(set(columns_to_drop))
        
        return df.drop(columns_to_drop)

    def convert_employment_length(self, value: str) -> float:
        '''
        Converts employment length from string format to a float.
        It handles special cases such as '< 1 year' and '10+ years', and returns None for missing values.

        Args:
            value (str): The employment length as a string.

        Returns:
            float: The employment length in years, or None if not applicable.
        '''

        if value is None or value == 'n/a':
            return None
        if value == '< 1 year':
            return 0.5
        if value == '10+ years':
            return 10.0
        return float(value.split()[0])

    def clean_lending_club_data(self):
        '''
        Clean and preprocess the Lending Club dataset.

        - Identifies and removes columns that are entirely null or contain date strings.
        - Converts date columns to datetime objects and adds derived features.
        - Converts and engineers other features (e.g., employment length, grade).
        - Drops undesired columns.
        - Saves the cleaned data and feature metadata for later use.
        '''

        df = pl.read_csv('temp.csv')

        null_columns = []
        date_columns = []
        
        #Identify null and date columns
        for col in df.columns:
            # Check if column is all null
            if df[col].is_null().all():
                null_columns.append(col)
                continue
            
            #Identify date columns
            sample = df[col].drop_nulls().sample(1)
            if self.is_date_value(str(sample[0])):
                date_columns.append(col)

        df = df.drop(null_columns)

        #Convert date columns to datetime
        if date_columns:
            df = df.with_columns([
                pl.col(col)
                .str.to_lowercase()
                .map_elements(self.parse_date_value, return_dtype=datetime)
                .alias(col)
                for col in date_columns
            ])

            #Include month and year columns for each date column
            expressions = []
            for col in date_columns:
                expressions.extend([
                    (pl.col(col).cast(pl.Int64)/pl.lit(2.628e+15)).alias(f'{col}_unicode_month')
                ])
            df = df.with_columns(expressions)

        #Ensures ordinal data is stored as integers while numerical data are floats
        df = df.with_columns([
            pl.col('term').str.extract(r'(\d+)').cast(pl.Int64).alias('term_months'),
            pl.col('emp_length').map_elements(self.convert_employment_length, return_dtype=pl.Float64).alias('employment_years'),
            pl.col('grade').str.to_uppercase().map_elements(lambda x: ord(x) - 64, return_dtype=pl.Int64).alias('grade'),
            pl.col('debt_settlement_flag').map_elements(lambda x: 1 if x == 'Y' else 0, return_dtype=pl.Int64).alias('debt_settlement_flag'),
            pl.col('orig_projected_additional_accrued_interest').cast(pl.Float64).alias('orig_projected_additional_accrued_interest')
            ]).drop(['emp_length', 'term'])
        
        df = self.drop_undesired_columns(df)

        os.remove('temp.csv')#Removes temporary file

        #Seperates the different columns by their data type
        standard_scaler_columns = [column for column, dtype in df.schema.items() 
                           if dtype==pl.Float64]
        min_max_scaler_columns = [column for column, dtype in df.schema.items() 
                           if dtype==pl.Int64]
        categorical_columns = [col for col, dtype in df.schema.items() 
                               if dtype == pl.Utf8]

        features_dict = {
            'standard_scaler': standard_scaler_columns,
            'min_max_scaler': min_max_scaler_columns,
            'categorical': categorical_columns
        }#Feature dictionary used to initialise the transformer

        with open(self.features_path, 'w') as f:
            json.dump(features_dict, f, indent=2)

        df = df.with_columns([
            pl.col(col).fill_null('missing') for col in categorical_columns
        ])#Ensures all missing values are represented the same way

        df.write_csv(self.cleaned_csv_path)

    def get_data_by_date_range(self, start_date: datetime, end_date: datetime, date_column: str = 'issue_d', return_unlisted_columns: bool = False):
        '''
        Extracts the rows from the cleaned dataset that fall within a specified date range.

        All columns can be returned or only those used for modelling.

        Args:
            start_date (datetime): The start date (inclusive).
            end_date (datetime): The end date (inclusive).
            date_column (str): The column to filter by date.
            return_unlisted_columns (bool): If True, return all columns; otherwise, drop datetime columns.

        Returns:
            pl.DataFrame: The filtered DataFrame.
        '''
        lf = pl.scan_csv(
            self.cleaned_csv_path, 
            low_memory=True,
            try_parse_dates=True)#Lazy frame is used for efficiency
        
        filtered_lf = lf.filter(
            pl.col(date_column).is_between(start_date, end_date)
        )

        if return_unlisted_columns:
            return filtered_lf.collect()
        else:
            df = filtered_lf.collect()
            df = filtered_lf.collect()

            datetime_columns = [column for column in df.columns if df[column].dtype == pl.Datetime]

            return df.drop(datetime_columns)#Dates are stored both through their unicode representation and as a datetime, the datetime is only required for filtering, it's disgarded for modelling

    def get_train_data(self, start_date: datetime, end_date: datetime)-> tuple[np.ndarray, np.ndarray]:
        '''
        Returns transformed training data for the autoencoder.
        Extracts data within the specified date range, applies feature transformations, and returns the data along with a mask indicating non-missing values.

        Args:
            start_date (datetime): The start date for training data.
            end_date (datetime): The end date for training data.

        Returns:
            tuple: (transformed_data, missing_mask)
                transformed_data (np.ndarray): The transformed feature matrix.
                missing_mask (np.ndarray): Boolean mask where True indicates non-missing values.
        '''
        self.training_data_start_date = start_date
        self.training_data_end_date = end_date

        raw_data = self.get_data_by_date_range(start_date, end_date, date_column='issue_d', return_unlisted_columns=False)
        transformed_data = self.transformer.fit_transform(raw_data)
        
        missing_mask = ~np.isnan(transformed_data)
        transformed_data[~missing_mask] = 0.0

        return transformed_data, missing_mask

    def get_test_data(self, start_date: datetime, end_date: datetime)-> tuple[np.ndarray, np.ndarray]:
        '''
        Returns transformed test data for the autoencoder.

        Extracts data within the specified date range, applies the previously fitted feature transformations, and returns the data and mask.
        Ensures there is no overlap with the training data.

        Args:
            start_date (datetime): The start date for test data.
            end_date (datetime): The end date for test data.

        Returns:
            tuple: (transformed_data, not_null_mask)
                transformed_data (np.ndarray): The transformed feature matrix.
                not_null_mask (np.ndarray): Boolean mask where True indicates non-missing values.

        Raises:
            ValueError: If training data has not been set or if there is overlap with training data.
        '''

        if self.training_data_start_date is None or self.training_data_end_date is None:
            raise ValueError('Training data not set. Please call get_training_data first.')
        
        if end_date < self.training_data_start_date or start_date > self.training_data_end_date:
            raw_data = self.get_data_by_date_range(start_date, end_date, date_column='issue_d', return_unlisted_columns=False)
            transformed_data = self.transformer.transform(raw_data)
        
            not_null_mask = ~np.isnan(transformed_data)
            transformed_data[~not_null_mask] = 0.0

            return transformed_data, not_null_mask
            
        else:
            raise ValueError('There is an overlap between the training and test data.')
        
    def get_transformed_data_feature_names(self):
        '''
        Returns the names of the features after the column transformation is applied.

        Returns:
            np.ndarray: Array of feature names as output by the column transformer.
        '''
        return self.transformer.get_feature_names_out()
    
    def get_sigmoid_feature_mask(self, as_torch=False):
        '''
        Returns a mask indicating which features should use a sigmoid activation (i.e., one-hot or min-max scaled).

        Args:
            as_torch (bool): If True, return as a torch tensor; otherwise, as a numpy array.

        Returns:
            np.ndarray or torch.Tensor: Boolean mask for sigmoid features.
        '''
        mask = np.array([1 if ('one_hot_encoder' in name or 'min_max_scaler' in name) else 0
                         for name in self.transformer.get_feature_names_out()])
        
        if as_torch:
            return torch.tensor(mask, dtype=torch.bool)
        else:
            return mask
    
    def get_binary_feature_mask(self, as_torch=False):
        '''
        Returns a mask indicating which features are binary (i.e., one-hot encoded or flag features).

        Args:
            as_torch (bool): If True, return as a torch tensor; otherwise, as a numpy array.

        Returns:
            np.ndarray or torch.Tensor: Boolean mask for binary features.
        '''
        mask = np.array([1 if ('one_hot_encoder' in name or 'flag' in name) else 0
                         for name in self.transformer.get_feature_names_out()])
        if as_torch:
            return torch.tensor(mask,dtype=torch.bool)
        else:
            return mask

In [18]:
#|export

def to_torch_dataloader(data, not_null_mask, batch_size: int = 64):
    '''
    Takes numpy arrays for the data and the corresponding not-null mask, converts them to PyTorch tensors,
    and returns a DataLoader for batching and shuffling during model training.

    Args:
        data (np.ndarray): The feature matrix to be loaded.
        not_null_mask (np.ndarray): Boolean mask indicating non-missing values in the data.
        batch_size (int, optional): Number of samples per batch. Defaults to 64.

    Returns:
        DataLoader: A PyTorch DataLoader yielding batches of (data, mask) tuples.
    '''
    data_tensor = torch.FloatTensor(data)
    not_null_mask_tensor = torch.BoolTensor(not_null_mask)

    # Create a dataset
    dataset = TensorDataset(data_tensor, not_null_mask_tensor)

    # Create dataloaders
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#|test

data_handler = DataHandler(csv_path='../local_data/all_lending_club_loan_data_2007-2018.csv')

start = datetime(2010,1,1)
end = datetime(2010,12,31)
train_data, train_mask = data_handler.get_train_data(start, end)

start = datetime(2011,1,1)
end = datetime(2011,12,31)
test_data, test_mask = data_handler.get_test_data(start, end)

if (train_data.shape[1] != test_data.shape[1]):
    raise ValueError('Training and testing data have different numbers of features')#Checks the same transformation is being applied to both datasets

#Make into torch dataloader
data_loader = to_torch_dataloader(train_data, train_mask)

if type(data_loader) != DataLoader:
    raise ValueError('Data loader is not a DataLoader')#Checks data is being correctly stored as a dataloader

